In [1]:
import numpy as np
import wave
import sys
import pygame
from time import sleep

from PyQt5.QtWidgets import (QDialog, QApplication, QWidget,
                             QVBoxLayout, QHBoxLayout,
                             QDesktopWidget, QFileDialog,
                             QSlider, QPushButton, QLabel, 
                             QCheckBox, QLCDNumber)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
import matplotlib.pyplot as plt



class Equalizer_Main_Class(QDialog):
    def __init__(self):
        super().__init__()
        self.initUI()

        
    def initUI(self):
        ### Hyperparameters ###
        self.nlabels = 10
        
        self.checkboxes_lables = ['Хор', 'Клиппинг']
        self.btns_lables = ['Воспроизвести', 'Пауза', 'Остановить']
        self.app_name = 'Эквалайзер'
        
        self.sld_min = -50
        self.sld_max = 50
        self.sld_def = 0
        self.sld_interval = 10
        self.sld_step = 1
        #######################
        
        ### Global variables ###
        self.path_to_pull = None
        
        self.nchannels = None        # number of channels
        self.sampwidth = None        # number of bytes per sample
        self.framerate = None        # number of frames per second
        self.nframes = None          # total number of frames
        self.comptype = None         # compression type 
        self.compname = None         # compression type name
        self.elem_per_herz = None
        self.koeff = 1000            # коэффициент прореживания
        self.types = {}
        self.buffer_size = None
        self.buffer_cnt = 0
        
        self.music_is_playing = False
        
        self.min_freq = 0
        self.max_freq = None
        
        self.channels = []
        self.spectrum = None
        
        self.bands = [[], []]
        self.labels = []
        ########################
        
        ### Links ###
#         https://habrahabr.ru/post/113239/
#         http://old.pynsk.ru/posts/2015/Nov/09/matematika-v-python-preobrazovanie-fure/
#         http://www.7not.ru/articles/klip_cool.phtml
#         https://martinfitzpatrick.name/article/multithreading-pyqt-applications-with-qthreadpool/
        #############
        
        self.path_to_pull = QFileDialog.getOpenFileName(self, 'Выберите .wav файл')[0]
        
        self.pull_music()
        self.create_bands()
        self.create_lables()
        self.create_LCD_numbers()
        self.create_sliders()
        self.create_checkboxes()
        self.create_buttons()
#         self.statusBar()
        self.create_graphics()
        self.create_interface()
    
    
    def pull_music(self):
        wav = wave.open(self.path_to_pull, mode = 'r')
        self.types = {
            1: np.int8,
            2: np.int16,
            4: np.int32
        }

        (self.nchannels, self.sampwidth,
         self.framerate, self.nframes,
         self.comptype, self.compname) = wav.getparams()
        
        self.max_freq = self.framerate // 2
        self.buffer_size = self.framerate

#         all frames in byte string
        content = wav.readframes(self.nframes)

#         one-dimensional array of audio stream samples
        samples = np.fromstring(content, dtype = self.types[self.sampwidth])

#         nchannels-dimensional array of audio stream samples
        for i in range(self.nchannels):
            self.channels.append(samples[i::self.nchannels])
        self.channels = np.array(self.channels)
        
        pygame.mixer.pre_init(frequency = self.framerate,
                              size = -self.sampwidth * 8,
                              channels = self.nchannels)
        pygame.init()
        
        
    def create_bands(self):
        step = (self.max_freq - self.min_freq) // 2**self.nlabels
        
        self.bands[0].append(self.min_freq)
        self.bands[1].append(self.min_freq + step)
        
        for i in range(1, self.nlabels - 1):
            self.bands[0].append(self.bands[1][i - 1])
            self.bands[1].append(self.bands[0][i] + 2**i * step)
        
        self.bands[0].append(self.bands[1][self.nlabels - 2])
        self.bands[1].append(self.max_freq)

        for i in range(self.nlabels):
            self.labels.append(str(self.bands[0][i]) + ' - ' + str(self.bands[1][i]))
    
    
    def create_lables(self):
        self.label_1 = QLabel(self.labels[0], self)
        self.label_2 = QLabel(self.labels[1], self)
        self.label_3 = QLabel(self.labels[2], self)
        self.label_4 = QLabel(self.labels[3], self)
        self.label_5 = QLabel(self.labels[4], self)
        self.label_6 = QLabel(self.labels[5], self)
        self.label_7 = QLabel(self.labels[6], self)
        self.label_8 = QLabel(self.labels[7], self)
        self.label_9 = QLabel(self.labels[8], self)
        self.label_10 = QLabel(self.labels[9], self)
    
    
    def create_LCD_numbers(self):
        self.num_1 = QLCDNumber(self)
        self.num_2 = QLCDNumber(self)
        self.num_3 = QLCDNumber(self)
        self.num_4 = QLCDNumber(self)
        self.num_5 = QLCDNumber(self)
        self.num_6 = QLCDNumber(self)
        self.num_7 = QLCDNumber(self)
        self.num_8 = QLCDNumber(self)
        self.num_9 = QLCDNumber(self)
        self.num_10 = QLCDNumber(self)
    
    
    def create_sliders(self):
        self.sld_1 = QSlider(Qt.Vertical, self)
        self.sld_2 = QSlider(Qt.Vertical, self)
        self.sld_3 = QSlider(Qt.Vertical, self)
        self.sld_4 = QSlider(Qt.Vertical, self)
        self.sld_5 = QSlider(Qt.Vertical, self)
        self.sld_6 = QSlider(Qt.Vertical, self)
        self.sld_7 = QSlider(Qt.Vertical, self)
        self.sld_8 = QSlider(Qt.Vertical, self)
        self.sld_9 = QSlider(Qt.Vertical, self)
        self.sld_10 = QSlider(Qt.Vertical, self)
        
        self.sld_1.setMinimum(self.sld_min)
        self.sld_2.setMinimum(self.sld_min)
        self.sld_3.setMinimum(self.sld_min)
        self.sld_4.setMinimum(self.sld_min)
        self.sld_5.setMinimum(self.sld_min)
        self.sld_6.setMinimum(self.sld_min)
        self.sld_7.setMinimum(self.sld_min)
        self.sld_8.setMinimum(self.sld_min)
        self.sld_9.setMinimum(self.sld_min)
        self.sld_10.setMinimum(self.sld_min)
        
        self.sld_1.setMaximum(self.sld_max)
        self.sld_2.setMaximum(self.sld_max)
        self.sld_3.setMaximum(self.sld_max)
        self.sld_4.setMaximum(self.sld_max)
        self.sld_5.setMaximum(self.sld_max)
        self.sld_6.setMaximum(self.sld_max)
        self.sld_7.setMaximum(self.sld_max)
        self.sld_8.setMaximum(self.sld_max)
        self.sld_9.setMaximum(self.sld_max)
        self.sld_10.setMaximum(self.sld_max)
        
        self.sld_1.setValue(self.sld_def)
        self.sld_2.setValue(self.sld_def)
        self.sld_3.setValue(self.sld_def)
        self.sld_4.setValue(self.sld_def)
        self.sld_5.setValue(self.sld_def)
        self.sld_6.setValue(self.sld_def)
        self.sld_7.setValue(self.sld_def)
        self.sld_8.setValue(self.sld_def)
        self.sld_9.setValue(self.sld_def)
        self.sld_10.setValue(self.sld_def)
        
        self.sld_1.setFocusPolicy(Qt.StrongFocus)
        self.sld_2.setFocusPolicy(Qt.StrongFocus)
        self.sld_3.setFocusPolicy(Qt.StrongFocus)
        self.sld_4.setFocusPolicy(Qt.StrongFocus)
        self.sld_5.setFocusPolicy(Qt.StrongFocus)
        self.sld_6.setFocusPolicy(Qt.StrongFocus)
        self.sld_7.setFocusPolicy(Qt.StrongFocus)
        self.sld_8.setFocusPolicy(Qt.StrongFocus)
        self.sld_9.setFocusPolicy(Qt.StrongFocus)
        self.sld_10.setFocusPolicy(Qt.StrongFocus)
        
        self.sld_1.setTickPosition(QSlider.TicksBothSides)
        self.sld_2.setTickPosition(QSlider.TicksBothSides)
        self.sld_3.setTickPosition(QSlider.TicksBothSides)
        self.sld_4.setTickPosition(QSlider.TicksBothSides)
        self.sld_5.setTickPosition(QSlider.TicksBothSides)
        self.sld_6.setTickPosition(QSlider.TicksBothSides)
        self.sld_7.setTickPosition(QSlider.TicksBothSides)
        self.sld_8.setTickPosition(QSlider.TicksBothSides)
        self.sld_9.setTickPosition(QSlider.TicksBothSides)
        self.sld_10.setTickPosition(QSlider.TicksBothSides)
        
        self.sld_1.setTickInterval(self.sld_interval)
        self.sld_2.setTickInterval(self.sld_interval)
        self.sld_3.setTickInterval(self.sld_interval)
        self.sld_4.setTickInterval(self.sld_interval)
        self.sld_5.setTickInterval(self.sld_interval)
        self.sld_6.setTickInterval(self.sld_interval)
        self.sld_7.setTickInterval(self.sld_interval)
        self.sld_8.setTickInterval(self.sld_interval)
        self.sld_9.setTickInterval(self.sld_interval)
        self.sld_10.setTickInterval(self.sld_interval)
        
        self.sld_1.setSingleStep(self.sld_step)
        self.sld_2.setSingleStep(self.sld_step)
        self.sld_3.setSingleStep(self.sld_step)
        self.sld_4.setSingleStep(self.sld_step)
        self.sld_5.setSingleStep(self.sld_step)
        self.sld_6.setSingleStep(self.sld_step)
        self.sld_7.setSingleStep(self.sld_step)
        self.sld_8.setSingleStep(self.sld_step)
        self.sld_9.setSingleStep(self.sld_step)
        self.sld_10.setSingleStep(self.sld_step)
        
        self.sld_1.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_2.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_3.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_4.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_5.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_6.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_7.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_8.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_9.valueChanged[int].connect(self.sliderChangeValue)
        self.sld_10.valueChanged[int].connect(self.sliderChangeValue)
        
        self.sld_1.valueChanged[int].connect(self.num_1.display)
        self.sld_2.valueChanged[int].connect(self.num_2.display)
        self.sld_3.valueChanged[int].connect(self.num_3.display)
        self.sld_4.valueChanged[int].connect(self.num_4.display)
        self.sld_5.valueChanged[int].connect(self.num_5.display)
        self.sld_6.valueChanged[int].connect(self.num_6.display)
        self.sld_7.valueChanged[int].connect(self.num_7.display)
        self.sld_8.valueChanged[int].connect(self.num_8.display)
        self.sld_9.valueChanged[int].connect(self.num_9.display)
        self.sld_10.valueChanged[int].connect(self.num_10.display)
        
        self.old_value_sld1 = self.sld_def
        self.old_value_sld2 = self.sld_def
        self.old_value_sld3 = self.sld_def
        self.old_value_sld4 = self.sld_def
        self.old_value_sld5 = self.sld_def
        self.old_value_sld6 = self.sld_def
        self.old_value_sld7 = self.sld_def
        self.old_value_sld8 = self.sld_def
        self.old_value_sld9 = self.sld_def
        self.old_value_sld10 = self.sld_def
        
    
    def create_checkboxes(self):
        self.checkbox_1 = QCheckBox(self.checkboxes_lables[0], self)
        self.checkbox_2 = QCheckBox(self.checkboxes_lables[1], self)
        
        self.checkbox_1.toggle()
        self.checkbox_2.toggle()
        
        self.checkbox_1.stateChanged.connect(self.checkboxClicked)
        self.checkbox_2.stateChanged.connect(self.checkboxClicked)
        
        
    def create_buttons(self):
        self.btn_1 = QPushButton(self.btns_lables[0], self)
        self.btn_2 = QPushButton(self.btns_lables[1], self)
        self.btn_3 = QPushButton(self.btns_lables[2], self)
        
        self.btn_1.clicked.connect(self.buttonClicked)
        self.btn_2.clicked.connect(self.buttonClicked)
        self.btn_3.clicked.connect(self.buttonClicked)
        
        
    def create_graphics(self):
        figure_1 = plt.figure()
        self.figure_2 = plt.figure()
        
        self.canvas_1 = FigureCanvas(figure_1)
        self.canvas_2 = FigureCanvas(self.figure_2)
            
        self.toolbar_1 = NavigationToolbar(self.canvas_1, self)
        self.toolbar_2 = NavigationToolbar(self.canvas_2, self)

        figure_1.clear()
        self.figure_2.clear()
        
        ax_1 = figure_1.add_subplot(1, 1, 1)
        self.ax_2 = self.figure_2.add_subplot(1, 1, 1)
        
        ax_1.set_xlabel('Частота, Гц')
        self.ax_2.set_xlabel('Частота, Гц')
        figure_1.align_xlabels()
        self.figure_2.align_xlabels()
        
        ax_1.set_ylabel('Амплитуда')
        self.ax_2.set_ylabel('Амплитуда')
        figure_1.align_ylabels()
        self.figure_2.align_ylabels()
        
#         figure_1.gca(title = 'Оригинальный спектр')
#         self.figure_2.gca(title = 'Изменённый спектр')
        
#         вычисляем преобразование Фурье. Сигнал действительный,
#         поэтому надо использовать rfft, это быстрее, чем fft
        self.spectrum = np.fft.rfft(self.channels)
    
        self.elem_per_herz = self.spectrum.shape[1] // (self.max_freq - self.min_freq)
        
        ax_1.plot(np.fft.rfftfreq(self.nframes, 1./ self.framerate)[::self.koeff], 
                  np.abs(self.spectrum[0][::self.koeff]) / self.nframes)
        self.ax_2.plot(np.fft.rfftfreq(self.nframes, 1./ self.framerate)[::self.koeff],
                       np.abs(self.spectrum[0][::self.koeff]) / self.nframes)
        
        self.canvas_1.draw()
        self.canvas_2.draw()

        
    def create_interface(self):
        self.labels_box = QHBoxLayout()
        self.labels_box.addWidget(self.label_1)
        self.labels_box.addWidget(self.label_2)
        self.labels_box.addWidget(self.label_3)
        self.labels_box.addWidget(self.label_4)
        self.labels_box.addWidget(self.label_5)
        self.labels_box.addWidget(self.label_6)
        self.labels_box.addWidget(self.label_7)
        self.labels_box.addWidget(self.label_8)
        self.labels_box.addWidget(self.label_9)
        self.labels_box.addWidget(self.label_10)
        
        self.nums_box = QHBoxLayout()
        self.nums_box.addWidget(self.num_1)
        self.nums_box.addWidget(self.num_2)
        self.nums_box.addWidget(self.num_3)
        self.nums_box.addWidget(self.num_4)
        self.nums_box.addWidget(self.num_5)
        self.nums_box.addWidget(self.num_6)
        self.nums_box.addWidget(self.num_7)
        self.nums_box.addWidget(self.num_8)
        self.nums_box.addWidget(self.num_9)
        self.nums_box.addWidget(self.num_10)
        
        self.slds_box = QHBoxLayout()
        self.slds_box.addWidget(self.sld_1)
        self.slds_box.addWidget(self.sld_2)
        self.slds_box.addWidget(self.sld_3)
        self.slds_box.addWidget(self.sld_4)
        self.slds_box.addWidget(self.sld_5)
        self.slds_box.addWidget(self.sld_6)
        self.slds_box.addWidget(self.sld_7)
        self.slds_box.addWidget(self.sld_8)
        self.slds_box.addWidget(self.sld_9)
        self.slds_box.addWidget(self.sld_10)
        
        self.checks_and_btns_box = QHBoxLayout()
        self.checks_and_btns_box.addWidget(self.checkbox_1)
        self.checks_and_btns_box.addWidget(self.checkbox_2)
        self.checks_and_btns_box.addWidget(self.btn_1)
        self.checks_and_btns_box.addWidget(self.btn_2)
        self.checks_and_btns_box.addWidget(self.btn_3)
        
        self.graphs_box = QVBoxLayout()
        self.graphs_box.addWidget(self.toolbar_1)
        self.graphs_box.addWidget(self.canvas_1)
        self.graphs_box.addWidget(self.toolbar_2)
        self.graphs_box.addWidget(self.canvas_2)
        
        self.left_box = QVBoxLayout()
        self.left_box.addLayout(self.labels_box)
        self.left_box.addLayout(self.slds_box)
        self.left_box.addLayout(self.nums_box)
        
        self.right_box = QVBoxLayout()
        self.right_box.addLayout(self.checks_and_btns_box)
        self.right_box.addLayout(self.graphs_box)
        
        self.all_box = QHBoxLayout()
        self.all_box.addLayout(self.left_box)
        self.all_box.addLayout(self.right_box)
        
        self.setLayout(self.all_box)
        
        self.setWindowTitle(self.app_name)
        self.showMaximized()
        
        
    def sliderChangeValue(self, value):
        sender = self.sender()
        senders = {
            self.sld_1: 0,
            self.sld_2: 1,
            self.sld_3: 2,
            self.sld_4: 3,
            self.sld_5: 4,
            self.sld_6: 5,
            self.sld_7: 6,
            self.sld_8: 7,
            self.sld_9: 8,
            self.sld_10: 9
        }
        pos = senders[sender]
        old_values = {
            0: self.old_value_sld1,
            1: self.old_value_sld2,
            2: self.old_value_sld3,
            3: self.old_value_sld4,
            4: self.old_value_sld5,
            5: self.old_value_sld6,
            6: self.old_value_sld7,
            7: self.old_value_sld8,
            8: self.old_value_sld9,
            9: self.old_value_sld10
        }
        old_value = old_values[pos]
        
#         filter
        if (pos == 0):
            for i in range(self.elem_per_herz * self.bands[1][pos] + 1):
                self.spectrum[0][i] *= 10**((value - old_value) / 20)
                self.spectrum[1][i] *= 10**((value - old_value) / 20)
        elif (pos == 9):
            for i in range(self.elem_per_herz * self.bands[0][pos],
                           self.spectrum.shape[1]):
                self.spectrum[0][i] *= 10**((value - old_value) / 20)
                self.spectrum[1][i] *= 10**((value - old_value) / 20)
        else:
            for i in range(self.elem_per_herz * self.bands[0][pos],
                           self.elem_per_herz * self.bands[1][pos] + 1):
                self.spectrum[0][i] *= 10**((value - old_value) / 20)
                self.spectrum[1][i] *= 10**((value - old_value) / 20)
        
        if pos == 0:
            self.old_value_sld1 = value
        elif pos == 1:
            self.old_value_sld2 = value
        elif pos == 2:
            self.old_value_sld3 = value
        elif pos == 3:
            self.old_value_sld4 = value
        elif pos == 4:
            self.old_value_sld5 = value
        elif pos == 5:
            self.old_value_sld6 = value
        elif pos == 6:
            self.old_value_sld7 = value
        elif pos == 7:
            self.old_value_sld8 = value
        elif pos == 8:
            self.old_value_sld9 = value
        elif pos == 9:
            self.old_value_sld10 = value
        
        self.channels = (np.fft.irfft(self.spectrum)).astype(self.types[self.sampwidth])
       
        self.figure_2.clear()
        self.ax_2 = self.figure_2.add_subplot(1, 1, 1)
        self.ax_2.set_xlabel('Частота, Гц')
        self.figure_2.align_xlabels()
        self.ax_2.set_ylabel('Амплитуда')
        self.figure_2.align_ylabels()
        self.ax_2.plot(np.fft.rfftfreq(self.nframes, 1./ self.framerate)[::self.koeff],
                       np.abs(self.spectrum[0][::self.koeff]) / self.nframes)
        self.canvas_2.draw()

            
    def checkboxClicked(self, state):
        sender = self.sender()
#         if (sender.text() == 'Choir'):
#             if (state == Qt.Checked):
#                 self.setWindowTitle('')
#             else:
#                 self.setWindowTitle('')
#         elif (sender.text() == 'Clipping'):
#             if (state == Qt.Checked):
#                 self.setWindowTitle('')
#             else:
#                 self.setWindowTitle('')
            
        
    def buttonClicked(self):
        sender = self.sender()
        if (sender.text() == self.btns_lables[0]):
            if (self.music_is_playing == False):
                self.music_is_playing = True
                
                self.start()
            
        elif (sender.text() == self.btns_lables[1]):
            if (self.music_is_playing):
                self.music_is_playing = False
                self.proc_music.terminate()
            
        elif (sender.text() == self.btns_lables[2]):
            if (self.music_is_playing):
                self.music_is_playing = False
                self.proc_music.terminate()
                
            
            self.sld_1.setValue(self.sld_def)
            self.sld_2.setValue(self.sld_def)
            self.sld_3.setValue(self.sld_def)
            self.sld_4.setValue(self.sld_def)
            self.sld_5.setValue(self.sld_def)
            self.sld_6.setValue(self.sld_def)
            self.sld_7.setValue(self.sld_def)
            self.sld_8.setValue(self.sld_def)
            self.sld_9.setValue(self.sld_def)
            self.sld_10.setValue(self.sld_def)

        
#         self.statusBar().showMessage(sender.text() + ' was pressed')


    def progress_fn(self, n):
        print("%d%% done" % n)

    def execute_this_fn(self, progress_callback):
        for n in range(0, 5):
            time.sleep(1)
            progress_callback.emit(n*100/4)

        return "Done."

    def print_output(self, s):
        print(s)

    def thread_complete(self):
        print("THREAD COMPLETE!")


    def start(self):
            tmp_channels = []
            tmp_channels.append(self.channels[0][self.buffer_cnt * self.buffer_size:
                                             (self.buffer_cnt + 1) * self.buffer_size + 1:])
            tmp_channels.append(self.channels[1][self.buffer_cnt * self.buffer_size:
                                             (self.buffer_cnt + 1) * self.buffer_size + 1:])
            tmp_channels = np.array(tmp_channels)
            tmp_channels = np.ascontiguousarray(tmp_channels.T)
            tmp_sound = pygame.sndarray.make_sound(tmp_channels)
            
            sound = tmp_sound
            pygame.mixer.Sound.play(sound)
        
            start_pos = self.buffer_cnt
            for self.buffer_cnt in range(start_pos + 1, self.nframes // self.buffer_size):
                tmp_channels = []
                tmp_channels.append(self.channels[0][self.buffer_cnt * self.buffer_size:
                                                     (self.buffer_cnt + 1) * self.buffer_size + 1:])
                tmp_channels.append(self.channels[1][self.buffer_cnt * self.buffer_size:
                                                     (self.buffer_cnt + 1) * self.buffer_size + 1:])
                tmp_channels = np.array(tmp_channels)
                tmp_channels = np.ascontiguousarray(tmp_channels.T)
                tmp_sound = pygame.sndarray.make_sound(tmp_channels)
            
                while (pygame.mixer.get_busy()):
                    sleep(0.01)
                    
                sound = tmp_sound
                pygame.mixer.Sound.play(sound)
        
            tmp_channels = []
            tmp_channels.append(self.channels[0][self.buffer_cnt * self.buffer_size::])
            tmp_channels.append(self.channels[1][self.buffer_cnt * self.buffer_size::])
            tmp_channels = np.array(tmp_channels)
            tmp_channels = np.ascontiguousarray(tmp_channels.T)
            tmp_sound = pygame.sndarray.make_sound(tmp_channels)
        
            while (pygame.mixer.get_busy()):
                sleep(0.01)
             
            sound = tmp_sound
            pygame.mixer.Sound.play(sound)


    
if __name__ == '__main__':
    Equalizer = QApplication(sys.argv)
    Equalizer_Instance = Equalizer_Main_Class()
    sys.exit(Equalizer.exec_())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:188: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Multithreading with maximum 4 threads


AttributeError: 'MainWindow' object has no attribute 'recurring_timer'